In [1]:
# Importing necessary libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Flatten, Dense
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.utils import plot_model
from sklearn.utils.class_weight import compute_class_weight
from tensorflow.keras.callbacks import ModelCheckpoint

In [2]:
# Creating ImageDataGenerators
image_generator = ImageDataGenerator(
    rescale=1./255.,
    shear_range=0.1,
    zoom_range=0.3,
)
image_generator_test = ImageDataGenerator(
    rescale=1./255.
)

In [3]:
# Loading training data
train_data = image_generator.flow_from_directory(
    'data',
    target_size=(500, 500), # Image size 500x500
    color_mode='grayscale', # Using grayscale images
    class_mode='binary', # Binary classification
    batch_size=50
)

In [4]:
# Loading testing data
test_data = image_generator_test.flow_from_directory(
    'test_data',
    target_size=(500, 500),
    color_mode='grayscale',
    class_mode='binary',
    batch_size=50
)

In [5]:
# Building the model architecture
model = Sequential()
model.add(Conv2D(32, (3, 3), activation='relu', input_shape=(500, 500, 1)))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(32, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPool2D(pool_size=(2, 2)))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

In [6]:
# Compiling the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [7]:
# Displaying the model architecture
plot_model(model, show_shapes=True, show_layer_names=False)

In [8]:
# Calculating class weights
# because we have low image --> we can set weight to classes --> because we have class 1 more than class 0 --> we set hight weight to class 0  
weights = compute_class_weight('balanced', np.unique(train_data.classes), train_data.classes)
class_weights = dict(zip(np.unique(train_data.classes), weights))

In [9]:
# Displaying the calculated class weights
class_weights

In [ ]:
# Setting up Early Stopping
# it code check model step by step and when loss does't decrease or doesn't much differend --> stop model (based on val_loss, mode='min'= we need min of val_loss, patience=3 how many step patience if val_loss doesn;t decrese )
es = EarlyStopping(monitor='val_loss', mode='min', patience=3) # Monitoring validation loss and stopping when it doesn't decrease
# Setting up Reduce Learning Rate on Plateau
# we use this for example if val_loss does't decrease or doesn't much differend --> don't update the weights
rlrop = ReduceLROnPlateau(monitor='val_loss', patience=2, factor=0.2, min_lr=0.001) # Reducing learning rate when validation loss doesn't decrease
# Setting up Model Checkpoint
 # if we want find best mode ( for example each epoch that have min val_loss --> save it for me) in each epoch and save that epoch to a file
mch = ModelCheckpoint('chestxray.h5', monitor='val_loss', mode='min', save_best_only=True) # Saving the best model based on validation loss

In [11]:
# Training the model
history = model.fit_generator(train_data, steps_per_epoch=len(train_data), epochs=30, validation_data=test_data, validation_steps=len(train_data), callbacks=[es, rlrop, mch], class_weight=class_weights)

In [12]:
# Plotting training and validation accuracy
plt.plot(history.history['accuracy'], color='blue') # Plotting training accuracy
plt.plot(history.history['val_accuracy'], color='green') # Plotting validation accuracy

In [13]:
# Plotting training and validation loss
plt.plot(history.history['loss'], color='blue') # Plotting training loss
plt.plot(history.history['val_loss'], color='green') # Plotting validation loss